In [1]:
import math
import os 
import numpy as np
import pandas as pd
import sqlite3

class SupportCalculations(object):
    
    conn = sqlite3.connect('tanks-4-09-data.db')
    
    def __init__(self, 
                 location, 
                 productMix, 
                 shellShade, 
                 shellCondition, 
                 roofShade, 
                 roofCondition, 
                 isShellInsulated, 
                 isRoofInsulated, 
                 shellHeight, 
                 tankDiameter,               
                 tankName,
                 assetNumber):
        
        self.location = location
        self.productMix = productMix
        self.shellShade = shellShade
        self.shellCondition = shellCondition
        self.roofShade = roofShade
        self.roofCondition = roofCondition
        self.isShellInsulated = isShellInsulated
        self.isRoofInsulated = isRoofInsulated
        self.shellHeight = shellHeight
        self.tankDiameter = tankDiameter
        self.tankName = tankName
        self.assetNumber = assetNumber
        
    def resultsShellRoofData(self):
        
        queryStringShell1 = " FROM solar_abs_data WHERE Surface_Color == " + "'" + str(self.shellShade) + "'"
        queryStringShell = "SELECT " + str(self.shellCondition) + queryStringShell1
        sqlQueryqueryStringShell = pd.read_sql(queryStringShell, self.conn)
        alphaS_shell = sqlQueryqueryStringShell.iloc[0][self.shellCondition]

        queryStringRoof1 = " FROM solar_abs_data WHERE Surface_Color == " + "'" + str(self.roofShade) + "'"
        queryStringRoof = "SELECT " + str(self.roofCondition) + queryStringRoof1
        sqlQueryqueryStringRoof = pd.read_sql(queryStringRoof, self.conn)
        alphaR_roof = sqlQueryqueryStringRoof.iloc[0][self.roofCondition]
        
        return {'alphaS_shell': alphaS_shell, 'alphaR_roof': alphaR_roof}
        
    def SQLResultsLocation(self):
        
        locationString1 = self.location
        locationString2 = "SELECT * FROM revised_met_data WHERE location == "
        locationString3 = "'" + str(locationString1) + "'"
        queryStringLocation = locationString2 + locationString3
        sqlDataLocation = pd.read_sql(queryStringLocation, self.conn)
        
        return sqlDataLocation    

    def SQLResultsProductData(self):
                      
        _productMix = self.productMix
        
        if len(_productMix) == 1:
                
            if _productMix[0]['productClass'] in ['Crude Oils']:
                 #Does not accept mixture
                if _productMix[0]['productName'] in ['Crude oil (RVP 5)']:

                    _queryStringProduct1 = "SELECT * FROM chemical_data " 
                    _queryStringProduct2 = "WHERE CATEGORY == " + "'"  
                    _queryStringProduct3 = str(_productMix[0]['productClass']) + "'"
                    _queryStringProduct = _queryStringProduct1 + _queryStringProduct2 + _queryStringProduct3
                    sqlDataProduct = pd.read_sql(_queryStringProduct, self.conn)

                    return sqlDataProduct 

                else:
                    raise ValueError('Assumption Invalid')

            elif _productMix[0]['productClass'] in ['Petroleum Distillates']:
                 #Does not accept mixture
                if _productMix[0]['productName'] in ['Gasoline (RVP 10)', 'Gasoline (RVP 13)', 'Gasoline (RVP 7)',
                                              'Gasoline (RVP 6)', 'Gasoline (RVP 8)', 'Gasoline (RVP 9)',
                                              'Gasoline (RVP 11)', 'Gasoline (RVP 12)', 'Gasoline (RVP 7.8)',
                                              'Gasoline (RVP 8.3)', 'Gasoline (RVP 11.5)','Gasoline (RVP 13.5)']:

                    _queryStringProduct1 = "SELECT * FROM chemical_data " 
                    _queryStringProduct2 = "WHERE CATEGORY == " + "'" 
                    _queryStringProduct3 = str(_productMix[0]['productClass']) + "'"
                    _queryStringProduct4 = " AND NAME == " +  "'" + str(_productMix[0]['productName']) + "'"
                    _queryStringProduct = _queryStringProduct1 + _queryStringProduct2 + _queryStringProduct3 + _queryStringProduct4

                    sqlDataProduct = pd.read_sql(_queryStringProduct, self.conn)

                    return sqlDataProduct
                
                else:
                    raise ValueError('Assumption Invalid')
            
            else:
                raise ValueError('Assumption Invalid')
        
        elif _productMix[0]['productClass'] in ['Organic Liquids']:
            
            #Accepts Mixture
            
            nameList = [each['productName'] for each in _productMix]
            compList = [each['composition'] for each in _productMix]
            
            if sum(compList) >= 0.999999:

                strBuild = "("
                for idx, each in enumerate(nameList[:-1]):
                    strBuild += "'" + each + "'"  + ', '
                whereString = strBuild + "'" + nameList[-1]+ "'" + ")"

                queryStringProductOL1 = "SELECT * FROM chemical_data "
                queryStringProductOL2 = "WHERE name IN " + whereString
                queryStringProductOL3 = queryStringProductOL1 + queryStringProductOL2
                _sqlDataProduct = pd.read_sql(queryStringProductOL3, self.conn)

                sqlDataProduct = _sqlDataProduct.assign(composition = compList)

            return sqlDataProduct
        
        else:
            raise ValueError('Assumption Invalid Mixture')

            
    def locationData(self):
    
        locationData = self.SQLResultsLocation()
        
        alphaS_shell = self.resultsShellRoofData()['alphaS_shell']
        alphaR_roof = self.resultsShellRoofData()['alphaR_roof']
    
        tax_maxAmbientTemp_R = locationData[locationData['symbol'] == 'TAX'].values.tolist()[0][-1] + 460.67
        tan_minAmbientTemp_R = locationData[locationData['symbol'] == 'TAN'].values.tolist()[0][-1] + 460.67
        data_list_wind = locationData[locationData['symbol'] == 'V'].values.tolist()[0][-1]
        i_solarInsulation = locationData[locationData['symbol'] == 'I'].values.tolist()[0][-1]
        atmPressure = locationData[locationData['symbol'] == 'PA'].values.tolist()[0][-1]
        
        # Equation 1.11. Assumed to be the difference between the annual value.
        delta_ta_R = (tax_maxAmbientTemp_R - tan_minAmbientTemp_R)
        
        # Equation 1.30, value is in Rankine. Assumed to be the sum 
        #    of the annual values divided by 2
        taa_averageDailyTemp = (tan_minAmbientTemp_R + tax_maxAmbientTemp_R) * (1/2.)

        # Equation 1.31. Assumes that the tank is not insulated. 
        # Not sure about tb for fully insulated or partially insulated tanks.
        tb_liquidBulkTemp = taa_averageDailyTemp + (0.003*alphaS_shell*i_solarInsulation)
        
        return {'city': self.location,
                'shell_shade': self.shellShade,
                'shell_condition': self.shellCondition,
                'roof_shade': self.roofShade,
                'roof_condition': self.roofCondition,
                'isShellInsulated': self.isShellInsulated, 
                'isShellInsulated':self.isShellInsulated,
                'tax_maxAmbientTemp_R': tax_maxAmbientTemp_R, 
                'tan_minAmbientTemp_R': tan_minAmbientTemp_R, 
                'wind_speed': data_list_wind,
                'delta_ta_R':delta_ta_R, 
                'alphaS_shell': alphaS_shell, 
                'alphaR_roof': alphaR_roof, 
                'i_solarInsulation': i_solarInsulation,
                'taa_averageDailyTemp': taa_averageDailyTemp, 
                'atmPressure': atmPressure, 
                'tb_liquidBulkTemp': tb_liquidBulkTemp,                 
                'version': '06/2020, Table: 7.1-7'}    
    
    def temperatureCalculations(self):
        
        locationTempData = self.locationData()
    
        #Fully Insulated
        if self.isShellInsulated == True and self.isRoofInsulated == True:
            
            # See Equation 1-5, note 1 for a fully insulated tank.
            # Assumes no cyclic heating of bulk liquid.
            deltatv_averageDailyVaporTempRange = 0
            
            # Equation 1-22, note 3, which references note 5.
            # TODO: Look into this issue for tb.
            # Issue here is that note 5 uses tb for a non-insulated tank, 
            # and we are working with a fully insulated tank.
            tla_averageDailyLiquidTemp = locationTempData['tb_liquidBulkTemp']
        
        #Partially Insulated
        elif self.isShellInsulated == True and self.isRoofInsulated == False:
            
            # Equation 1.8.
            # `delta_ta_R` is assumed to be in Rankine
            deltatv_averageDailyVaporTempRange = (0.6*locationTempData['delta_ta_R']) +\
                                                 (0.02*locationTempData['alphaR_roof'] *\
                                                  locationTempData['i_solarInsulation'])
            
            # Equation 1.29 for a partially insulated tank.
            # Again, the tb value is for a non-insulated tank.
            tla_averageDailyLiquidTemp = 0.3*locationTempData['taa_averageDailyTemp'] +\
                                            0.7*locationTempData['tb_liquidBulkTemp'] +\
                                            0.005*locationTempData['alphaR_roof'] *\
                                            locationTempData['i_solarInsulation']
        
        #Uninsulated
        elif self.isShellInsulated == False and self.isRoofInsulated == False:
            
            # Equation 1.6 for uninsulated tank.
            hs_d = self.shellHeight / self.tankDiameter
            
            del_tv_1 = (2.2 * hs_d) + 1.9
            del_tv_2 = 0.042*locationTempData['alphaR_roof']*locationTempData['i_solarInsulation']
            del_tv_3 = 0.026*hs_d*locationTempData['alphaS_shell']*locationTempData['i_solarInsulation']
            del_tv_4 = locationTempData['delta_ta_R']
            del_tv_5 = (1 - ((0.8)/(del_tv_1)))*del_tv_4
            del_tv_6 = (del_tv_2 + del_tv_3) / del_tv_1
            
            deltatv_averageDailyVaporTempRange = del_tv_5 + del_tv_6
            
            # Equation 1.27 for uninsulated tank.
            tla_1 = (4.4 * hs_d) + 3.8
            tla_2_1 = 0.021*locationTempData['alphaR_roof']*locationTempData['i_solarInsulation']
            tla_2_2 = 0.013*hs_d*locationTempData['alphaS_shell']*locationTempData['i_solarInsulation']
            
            pt_1 = (0.5-(0.8/tla_1))*locationTempData['taa_averageDailyTemp']
            pt_2 = (0.5+(0.8/tla_1))*locationTempData['tb_liquidBulkTemp']
            pt_3 = (tla_2_1 + tla_2_2) / tla_1
            
            tla_averageDailyLiquidTemp = pt_1 + pt_2 + pt_3
            
        else:
            raise ValueError('Assumption Invalid')
            
        # Figure 7.1-17, values are in Rankine
        tlx_averageDailyMaxLiqSurfaceTemp_R = tla_averageDailyLiquidTemp +\
                                            (0.25 * deltatv_averageDailyVaporTempRange)
        
        tln_averageDailyMinLiqSurfaceTemp_R = tla_averageDailyLiquidTemp -\
                                            (0.25 * deltatv_averageDailyVaporTempRange)
        
        return {'tlx_R': tlx_averageDailyMaxLiqSurfaceTemp_R,
                'tln_R': tln_averageDailyMinLiqSurfaceTemp_R, 
                'tla_R': tla_averageDailyLiquidTemp, 
                'delta_tv': deltatv_averageDailyVaporTempRange}
    
    def organicLiquidMixtureVaporMolecularWeight(self):
        
        _productMix = self.productMix
        
        if _productMix[0]['productClass'] in ['Organic Liquids']:
        
            productDataOL = self.SQLResultsProductData()
            tempData = self.temperatureCalculations()

            check_A = productDataOL['VP_COEF_A'].values.tolist()
            check_B = productDataOL['VP_COEF_B'].values.tolist()
            check_C = productDataOL['VP_COEF_C'].values.tolist()

            if 0 not in check_A:
                if 0 not in check_B:
                    if 0 not in check_C:

                        # tlx_averageDailyMaxLiqSurfaceTemp_R
                        productDataOL['log_pva'] = productDataOL['VP_COEF_A'] - ((productDataOL['VP_COEF_B']) /\
                                                                                       (((tempData['tla_R'] - 491.7)/1.8) + productDataOL['VP_COEF_C']))
                        
                        productDataOL['log_pvx'] = productDataOL['VP_COEF_A'] - ((productDataOL['VP_COEF_B']) /\
                                                                                       (((tempData['tlx_R'] - 491.7)/1.8) + productDataOL['VP_COEF_C']))
                        
                        productDataOL['log_pvn'] = productDataOL['VP_COEF_A'] - ((productDataOL['VP_COEF_B']) /\
                                                                                       (((tempData['tln_R'] - 491.7)/1.8) + productDataOL['VP_COEF_C']))
                        
                        productDataOL['pva'] = 10**(productDataOL['log_pva']) / 51.7149
                        productDataOL['pvx'] = 10**(productDataOL['log_pvx']) / 51.7149
                        productDataOL['pvn'] = 10**(productDataOL['log_pvn']) / 51.7149
                        
                        productDataOL['moles'] = 100*productDataOL['composition'] / productDataOL['MOLWT']
                        productDataOL['mol_frac'] =  productDataOL['moles'] / sum(productDataOL['moles'].values.tolist())
                        productDataOL['frac_pva'] = productDataOL['pva'] * productDataOL['composition']
                        productDataOL['frac_pvx'] = productDataOL['pvx'] * productDataOL['composition']
                        productDataOL['frac_pvn'] = productDataOL['pvn'] * productDataOL['composition']
                        
                        productDataOL['partial_pressure_pva'] = productDataOL['mol_frac'] * productDataOL['pva']
                        productDataOL['partial_pressure_pvx'] = productDataOL['mol_frac'] * productDataOL['pvx']
                        productDataOL['partial_pressure_pvn'] = productDataOL['mol_frac'] * productDataOL['pvn']
                        
                        total_pva = sum(productDataOL['partial_pressure_pva'].values.tolist())
                        total_pvx = sum(productDataOL['partial_pressure_pvx'].values.tolist())
                        total_pvn = sum(productDataOL['partial_pressure_pvn'].values.tolist())
                        
                        productDataOL['y_i_pva'] = productDataOL['partial_pressure_pva'] / total_pva
                        productDataOL['y_i_pvx'] = productDataOL['partial_pressure_pvx'] / total_pvx
                        productDataOL['y_i_pvn'] = productDataOL['partial_pressure_pvn'] / total_pvn
                        
                        productDataOL['mv_i_pva'] = productDataOL['MOLWT'] * productDataOL['y_i_pva'] 
                        productDataOL['mv_i_pvx'] = productDataOL['MOLWT'] * productDataOL['y_i_pvx'] 
                        productDataOL['mv_i_pvn'] = productDataOL['MOLWT'] * productDataOL['y_i_pvn'] 
        
                        vapor_mw = sum(productDataOL['mv_i_pva'].values.tolist())
            
                    else:
                        raise ValueError('Value C Not Present in Data')
                else:
                    raise ValueError('Value B Not Present in Data')
            else: 
                raise ValueError('Value A Not Present in Data')

        else:
            raise ValueError('Grant')
        
        return {'pva': total_pva, 
                'plx': total_pvx, 
                'pln': total_pvn, 
                'delta_pv': total_pvx - total_pvn,
                'vapor_mw': vapor_mw, 
                'rvp': None}    
    
    def crudePetVaporMolecularWeight(self):
        
        _productMix = self.productMix
        
        if _productMix[0]['productClass'] in ['Crude Oils', 'Petroleum Distillates']:
            
            
            _temperatureCalculations = self.temperatureCalculations()
            _SQLResultsProductData = self.SQLResultsProductData()

            rvp = _SQLResultsProductData['REID']

            const_a = 12.82 - 0.9672*math.log(rvp)
            const_b = 7261 - 1216*math.log(rvp)

            constDict =  {'const_a': const_a, 'const_b': const_b}
            val_pva = constDict['const_a'] - ( (constDict['const_b']) / _temperatureCalculations['tla_R'] )
            val_pvx = constDict['const_a'] - ( (constDict['const_b']) / _temperatureCalculations['tlx_R'] )
            val_pvn = constDict['const_a'] - ( (constDict['const_b']) / _temperatureCalculations['tln_R'] )

            pva_trueVaporPressure = math.exp(val_pva)
            pvx_vapPressAveDailyMaxSurfaceTemp = math.exp(val_pvx)
            pvn_vapPressAveDailyMinSurfaceTemp = math.exp(val_pvn)
            vapor_mw = _SQLResultsProductData['VP_MOLWT'].values.tolist()[0]
            rvp = _SQLResultsProductData['REID'].values.tolist()[0]

            return {'pva': pva_trueVaporPressure, 
                    'plx': pvx_vapPressAveDailyMaxSurfaceTemp, 
                    'pln': pvn_vapPressAveDailyMinSurfaceTemp, 
                    'delta_pv': pvx_vapPressAveDailyMaxSurfaceTemp - pvn_vapPressAveDailyMinSurfaceTemp,
                    'vapor_mw': vapor_mw, 
                    'rvp': rvp}
        
        else:
            raise ValueError('Incorrect Assumption')
               
class FixedRoofTank(SupportCalculations):
    
    conn = sqlite3.connect('tanks-4-09-data.db')
    
    def __init__(self, 
                 location, 
                 productMix, 
                 shellShade, 
                 shellCondition, 
                 roofShade, 
                 roofCondition, 
                 isShellInsulated, 
                 isRoofInsulated, 
                 shellHeight, 
                 tankDiameter,
                 tankName,
                 assetNumber,
                 tankOrientation, 
                 isAverageLiquidHeightKnown, 
                 isTankDomeRoofRadiusKnown,
                 tankLength, 
                 roofType, 
                 tankDomeRoofRadius, 
                 roofSlope, 
                 breatherPressureSetting, 
                 breatherVaccumSetting, 
                 throughput, 
                 isTankOnVaporBalance, 
                 breatherVentSetting, 
                 averageLiquidHeight):
        
        super().__init__(
                 location, 
                 productMix, 
                 shellShade, 
                 shellCondition, 
                 roofShade, 
                 roofCondition, 
                 isShellInsulated, 
                 isRoofInsulated, 
                 shellHeight, 
                 tankDiameter,                 
                 tankName,
                 assetNumber,)
        
        self.tankOrientation = tankOrientation
        self.isAverageLiquidHeightKnown = isAverageLiquidHeightKnown
        self.isTankDomeRoofRadiusKnown = isTankDomeRoofRadiusKnown
        self.tankLength = tankLength
        self.roofType = roofType 
        self.tankDomeRoofRadius = tankDomeRoofRadius
        self.roofSlope = roofSlope
        self.breatherPressureSetting = breatherPressureSetting
        self.breatherVaccumSetting = breatherVaccumSetting
        self.throughput = throughput
        self.isTankOnVaporBalance = isTankOnVaporBalance
        self.breatherVentSetting = breatherVentSetting
        self.averageLiquidHeight = averageLiquidHeight

    
    def stockVaporDensity(self):
        
        _locationData = self.locationData()
        _productMix = self.productMix
        hs_d = self.shellHeight / self.tankDiameter
        
        #Fully Insulated
        if self.isShellInsulated == True and self.isRoofInsulated == True:
            
            tv_averageVaporTemp = _locationData['tb_liquidBulkTemp']
            
        #Partially Insulated
        elif self.isShellInsulated == True and self.isRoofInsulated == False:
            
            tv_averageVaporTemp = 0.6 * _locationData['taa_averageDailyTemp'] + \
                                    0.4*_locationData['tb_liquidBulkTemp'] + \
                                    0.01 * _locationData['alphaR_roof'] * _loctionData['i_solarInsulation']
            
        #Partially Insulated
        elif self.isShellInsulated == False and self.isRoofInsulated == False:
            wv_pt_1 = (2.2 * hs_d + 1.1) * _locationData['taa_averageDailyTemp']
            wv_pt_2 = 0.8 * _locationData['tb_liquidBulkTemp']
            wv_pt_3 = 0.021 * _locationData['alphaR_roof'] * _locationData['i_solarInsulation']
            wv_pt_4 = 0.013 * hs_d * _locationData['alphaS_shell'] * _locationData['i_solarInsulation']
            wv_pt_5 = 2.2 * hs_d + 1.9
            
            tv_averageVaporTemp = (wv_pt_1 + wv_pt_2 + wv_pt_3 + wv_pt_4) / wv_pt_5
            
        else:
            raise ValueError('Error')
            
        if _productMix[0]['productClass'] in ['Crude Oils', 'Petroleum Distillates']:
            pva = self.crudePetVaporMolecularWeight()['pva']
            mw = self.crudePetVaporMolecularWeight()['vapor_mw']
            vaporDensity = (pva * mw) / (10.731 * tv_averageVaporTemp)
        
        elif _productMix[0]['productClass'] in ['Organic Liquids']:
            
            pva = self.organicLiquidMixtureVaporMolecularWeight()['pva']
            mw = self.organicLiquidMixtureVaporMolecularWeight()['vapor_mw']
            vaporDensity = (pva * mw) / (10.731 * tv_averageVaporTemp)
        
        else:
            raise ValueError('Grant')
            
        return vaporDensity
    
    
    def vaporSpaceOutage(self):
        '''
        Done - Check
        Equation 1-16. 
        Uses Notes 1, 2 and Equations 1-17 - 1-20.
        '''

        # Assumptions made within the function.
        tankShellRadius = self.tankDiameter / 2.
        
        if tankShellRadius >= 0.8*self.tankDiameter or\
            tankShellRadius <= 1.2*self.tankDiameter:
            pass
    
        # Assumed to be half of the shell height.
        if self.isAverageLiquidHeightKnown == True:
            aveLiquidLevel = self.averageLiquidHeight
            
        elif self.isAverageLiquidHeightKnown == False:
            aveLiquidLevel = self.shellHeight / 2.
            
        else:
            raise ValueError('Invalid Assumption')
            
        # hro_roof_outage
        if self.roofType == 'Dome':

            if self.isTankDomeRoofRadiusKnown == True:
                
                valueSqRoot = (self.tankDomeRoofRadius**2) -\
                              (tankShellRadius**2)
                
                if valueSqRoot >= 0:
                    #Equation 1.20.
                    roofHeight   =   self.tankDomeRoofRadius -\
                                        (valueSqRoot)**(0.5)
                
                else:
                    raise ValueError('Taking the negative of a square root')
                
                # Equation 1.19
                hro_roofOutage  =   roofHeight *\
                                    ( (1 / 2.) + ( (1 / 6.) *\
                                    (roofHeight / tankShellRadius)**2) )
            
            elif self.isTankDomeRoofRadiusKnown == False:
                
                #Why does this matter if the value is unknown?
                roofHeight = 0.268 * tankShellRadius 
                
                hro_roofOutage = 0.137 * tankShellRadius
            
            else:
                raise ValueError('Assumption Invalid')

        elif self.roofType == 'Cone':
            # Note 1 to determine hro_roof_outage for a cone roof.
            # Equation 1.17, and 1.18.
            hro_roofOutage = ( 1 / 3. ) * self.roofSlope * tankShellRadius
        
        else:
            raise ValueError('Incorrect Roof Type.')
        
        # hvo_vaporSpaceOutage
        if self.tankOrientation == 'Vertical':
            #Equation 1.16.
            hvo_vaporSpaceOutage =    self.shellHeight - \
                                        aveLiquidLevel + hro_roofOutage
            
            result = {'value': hvo_vaporSpaceOutage, 
                      'quantity': 'Vapor Space Outage (hvo)',
                      'equation': '1-16',
                      'version': '06/2020', 'elements': [{'hs': self.shellHeight, 
                                                          'hl': aveLiquidLevel, 
                                                          'hro': hro_roofOutage, 
                                                          'hr': self.roofSlope * tankShellRadius, 
                                                          'roof slope': self.roofSlope, 
                                                          'roof height': self.roofSlope * tankShellRadius,
                                                          'shell radius': tankShellRadius,
                                                          'shell height': self.shellHeight,
                                                          'liquid height': aveLiquidLevel,
                                                          'roof type': self.roofType,
                                                          'constant': None,
                                                          'd': self.tankDiameter,
                                                          'tank_orientation': self.tankOrientation,
                                                          'status': 'Done'}]} 
        elif self.tankOrientation == 'Horizontal':
            # See HVO, in Equation 1-16.
            hvo_vaporSpaceOutage = ( (math.pi) / 8. ) * self.tankDiameter
            
            result = {'value': hvo_vaporSpaceOutage, 
                      'quantity': 'Vapor Space Outage (hvo)',
                      'equation': '1-16',
                      'version': '06/2020', 
                      'elements': [{'hs': None, 
                                  'hl': None, 
                                  'hro': None, 
                                  'hr': None,
                                  'roof slope': None, 
                                  'roof height': None,
                                  'shell radius': None,
                                  'shell height': None,
                                  'liquid height': None,
                                  'roof type': None,
                                  'constant': 'pi/8',
                                  'd': self.tankDiameter,
                                  'tank_orientation': self.tankOrientation,
                                  'status': 'Done'}]} 
        
        else:
            raise ValueError('Incorrect Tank Orientation. \
                              Tank can either be Vertical \
                              or Horizontal.')
        
        return result
        
    def vaporSpaceVolume(self):
        '''
        Equation 1.3.
        '''
        if self.tankOrientation == 'Horizontal':

            effectiveDiameter = ( (self.tankLength * self.tankDiameter) /\
                                ( (1. / 4.) * (math.pi) ) ) ** (0.5)
            _tankDiameter = effectiveDiameter

        elif self.tankOrientation == 'Vertical':
            _tankDiameter = self.tankDiameter

        else:
            raise ValueError('Incorrect Tank Type')

        vv_vaporSpaceVolume =   (math.pi) * (1. / 4.) *\
                                (_tankDiameter ** 2) *\
                                self.vaporSpaceOutage()['value']

        return {'quantity': 'Tank Vapor Space Volume (vv)', 
                'equation': '1-3', 
                'value': vv_vaporSpaceVolume,
                'version': '06/2020',
                'notes': 'Uses effective diameter (eqn. 1-14) for horizontal tanks.',
                'elements': [{'constant': 'pi/4',
                              'tank orientation': self.tankOrientation,
                              'd': _tankDiameter, 
                              'hvo': self.vaporSpaceOutage()['value'], 
                              'status': 'Done'}]}
    
    def ventedVaporSpaceSatFactor(self):
        '''
        Partially Done - Outstanding item is to 
        compute pva (eqn 1-22) for mixtures.
        Equation 1-21.
        '''

        # Need to pass in a dict of all of the 
        # liquid components in the mixture, or a 
        # vapor pressure from a look up table.

        # TODO: Compute molecular weight of the vapor phase
        # TODO: Compuete total vapor pressure of the liquid.

        # Equation 1.22. See notes 1, and 2.
        # TODO: Build this part out for mixtures. What is here is for a single component.
        
        
        _productMix = self.productMix
        
        if _productMix[0]['productClass'] in ['Crude Oils', 'Petroleum Distillates']:
            vaporPresureArray = self.crudePetVaporMolecularWeight()
        
        elif _productMix[0]['productClass'] in ['Organic Liquids']:
            vaporPresureArray = self.organicLiquidMixtureVaporMolecularWeight()
        else:
            raise ValueError('Injcorrect Assumption')
        
        pva_vaporPressureAverageDaily = vaporPresureArray['pva']   

        # Equation 1.16
        hvo_vaporSpaceOutage = self.vaporSpaceOutage()['value']             

        ks_ventedVaporSpaceSatFactor = 1 / (1 + ( 0.053 *\
                                                 pva_vaporPressureAverageDaily *\
                                                 self.vaporSpaceOutage()['value'] ) )

        return {'quantity': 'Vented Vapor Saturation Factor (ks)', 
                'equation': '1-12', 
                'value': ks_ventedVaporSpaceSatFactor, 
                'version': '06/2020', 
                'notes': 'none', 
                'elements': [{'constant': 0.053, 
                              'pva': pva_vaporPressureAverageDaily, 
                              'hvo': hvo_vaporSpaceOutage, 
                              'status': 'Partially Done, need to include calculations for mixtures.'}]}
    
    def vaporSpaceExpansionFactor(self):
        '''
        Partially done -- Look into the value for tb 
        and the tank insulation, and the partial pressures. 
        Equation 1-5.
        
        Most of the values match the OK printout.
        
        Also, look into `delta_pv`, and need to fold in 
            logifc to pull in rvp based in user input.
    
        '''
        
        locationTempData = self.locationData()
        _productMix = self.productMix
        
        #Fully Insulated
        if self.isShellInsulated == True and self.isRoofInsulated == True:
            
            # See Equation 1-5, note 1 for a fully insulated tank.
            # Assumes no cyclic heating of bulk liquid.
            deltatv_averageDailyVaporTempRange = 0
            
            # Equation 1-22, note 3, which references note 5.
            # TODO: Look into this issue for tb.
            # Issue here is that note 5 uses tb for a non-insulated tank, 
            # and we are working with a fully insulated tank.
            tla_averageDailyLiquidTemp = locationTempData['tb_liquidBulkTemp']
        
        #Partially Insulated
        elif self.isShellInsulated == True and self.isRoofInsulated == False:
            
            # Equation 1.8.
            # `delta_ta_R` is assumed to be in Rankine
            deltatv_averageDailyVaporTempRange = (0.6*locationTempData['delta_ta_R']) +\
                                                 (0.02*locationTempData['alphaR_roof'] *\
                                                  locationTempData['i_solarInsulation'])
            
            # Equation 1.29 for a partially insulated tank.
            # Again, the tb value is for a non-insulated tank.
            tla_averageDailyLiquidTemp = 0.3*locationTempData['taa_averageDailyTemp'] +\
                                            0.7*locationTempData['tb_liquidBulkTemp'] +\
                                            0.005*locationTempData['alphaR_roof'] *\
                                            locationTempData['i_solarInsulation']
        
        #Uninsulated
        elif self.isShellInsulated == False and self.isRoofInsulated == False:
            
            # Equation 1.6 for uninsulated tank.
            hs_d = self.shellHeight / self.tankDiameter
            
            delta_tv_1 = (2.2 * hs_d) + 1.9
            delta_tv_2 = 0.042*locationTempData['alphaR_roof']*locationTempData['i_solarInsulation']
            delta_tv_3 = 0.026*hs_d*locationTempData['alphaS_shell']*locationTempData['i_solarInsulation']
            delta_tv_4 = locationTempData['delta_ta_R']
            
            delta_tv_5 = (1 - ((0.8)/(delta_tv_1)))* delta_tv_4
            delta_tv_6 = (delta_tv_2 + delta_tv_3) / delta_tv_1
            
            deltatv_averageDailyVaporTempRange = delta_tv_5 + delta_tv_6
            
            # Equation 1.27 for uninsulated tank.
            tla_1 = (4.4 * hs_d) + 3.8
            tla_2_1 = 0.021*locationTempData['alphaR_roof']*locationTempData['i_solarInsulation']
            tla_2_2 = 0.013*hs_d*locationTempData['alphaS_shell']*locationTempData['i_solarInsulation']
            
            pt_1 = (0.5-(0.8/tla_1))*locationTempData['taa_averageDailyTemp']
            pt_2 = (0.5+(0.8/tla_1))*locationTempData['tb_liquidBulkTemp']
            pt_3 = (tla_2_1 + tla_2_2) / tla_1
            
            tla_averageDailyLiquidTemp = pt_1 + pt_2 + pt_3
            
        else:
            raise ValueError('Assumption Invalid')
            
        # Figure 7.1-17, values are in Rankine
        tlx_averageDailyMaxLiqSurfaceTemp_R = tla_averageDailyLiquidTemp +\
                                            (0.25 * deltatv_averageDailyVaporTempRange)
        
        tln_averageDailyMinLiqSurfaceTemp_R = tla_averageDailyLiquidTemp -\
                                            (0.25 * deltatv_averageDailyVaporTempRange)
                
        # Only for Crude Oils, and Selected Petroleum Stocks
        if _productMix[0]['productClass'] in ['Crude Oils', 'Petroleum Distillates']:
            
            pva_trueVaporPressure = self.crudePetVaporMolecularWeight()['pva']
            pvx_vapPressAveDailyMaxSurfaceTemp = self.crudePetVaporMolecularWeight()['plx'] 
            pvn_vapPressAveDailyMinSurfaceTemp = self.crudePetVaporMolecularWeight()['pln'] 
        
        # TODO: Need to resolve this function for organic liquids.
        elif _productMix[0]['productClass'] in ['Organic Liquids']:
            
            pva_trueVaporPressure = self.organicLiquidMixtureVaporMolecularWeight()['pva']
            pvx_vapPressAveDailyMaxSurfaceTemp = self.organicLiquidMixtureVaporMolecularWeight()['plx'] 
            pvn_vapPressAveDailyMinSurfaceTemp = self.organicLiquidMixtureVaporMolecularWeight()['pln'] 
                 
        else:
            raise ValueError('Incorrect Assumption')
            
        pva_vaporPressureAverageDaily = pva_trueVaporPressure 
        
        # Equation 1.10.
        delta_pb = self.breatherPressureSetting - self.breatherVaccumSetting
        
        # Equation 1.9
        delta_pv = pvx_vapPressAveDailyMaxSurfaceTemp - pvn_vapPressAveDailyMinSurfaceTemp
        
        pt_4 = (deltatv_averageDailyVaporTempRange / tla_averageDailyLiquidTemp)
        pt_5 = locationTempData['atmPressure'] - pva_vaporPressureAverageDaily
        pt_6 = delta_pv - delta_pb

        ke_vaporSpaceExpansionFactor =  pt_4 + (pt_6 / pt_5)
        
        if ke_vaporSpaceExpansionFactor <= 1 and ke_vaporSpaceExpansionFactor > 0:
            _ke_vaporSpaceExpansionFactor = ke_vaporSpaceExpansionFactor
            
        else:
            raise ValueError('Assumption Invalid')
            
        return {'quantity': 'Vapor Space Expansion Factor (ke)', 
                'equation': '1-5',
                'version': '06/2020',
                'value': _ke_vaporSpaceExpansionFactor, 
                'elements': [{'delta_tv': deltatv_averageDailyVaporTempRange, 
                             'tla': tla_averageDailyLiquidTemp, 
                              'atm_pressure': locationTempData['atmPressure'], 
                              'pva': pva_vaporPressureAverageDaily,
                              'delta_pv': delta_pv, 
                              'delta_pb': delta_pb, 
                              'tlx': tlx_averageDailyMaxLiqSurfaceTemp_R, 
                              'tln': tln_averageDailyMinLiqSurfaceTemp_R, 
                              'pvx': pvx_vapPressAveDailyMaxSurfaceTemp, 
                              'pvn':pvn_vapPressAveDailyMinSurfaceTemp,
                              'product class': _productMix[0]['productClass'], 
                              'product name': _productMix[0]['productName'],
                              'status': 'None'}]}
    
    def standingLosses(self):
        '''
        Done -- Check.
        Equation 1.2.
        '''
        standingLoss = 365 *\
                        self.vaporSpaceVolume()['value'] *\
                        self.stockVaporDensity() *\
                        self.vaporSpaceExpansionFactor()['value'] *\
                        self.ventedVaporSpaceSatFactor()['value']

        return {'quantity': 'standingLoss', 
                'value': standingLoss,
                'equation': '1-2',
                'version': '06/2020',
                'elements': [{'constant': '365', 
                             'vv': self.vaporSpaceVolume()['value'] , 
                             'wv': self.stockVaporDensity(),
                             'ke': self.vaporSpaceExpansionFactor()['value'], 
                             'ks': self.ventedVaporSpaceSatFactor()['value'], 
                             'status': 'Done'}]}
    
    def workingLosses(self):

        '''
        Done -- Check.
        Equation 1-35.
        '''
        
        tempData = self.locationData()
        _productMix = self.productMix
        
        pa_atmosphericPressure = tempData['atmPressure']
        
        # ========================================================================= #
        
        # hqi_annualSumIncreases
        if self.tankOrientation == 'Horizontal':
            effectiveDiameter = ( (self.tankLength * self.tankDiameter) / \
                                 ( (1. / 4.) * (math.pi) ) ) ** (0.5)
            _tankDiameter = effectiveDiameter
        
        elif self.tankOrientation == 'Vertical':
            _tankDiameter = self.tankDiameter
        
        else:
            raise ValueError('Assumption Invalid')
        
        hqi_annualSumIncreases = (5.614 * self.throughput/42) /\
                                    (((math.pi) / 4.) * (_tankDiameter)**(2))
        
         # ========================================================================= #       
        
        # hlx_maximumLiquidHeight, and hln_minimumLiquidHeight
        # Assume that in both cases, the min. and max liquid level is not known. 
        if self.tankOrientation == 'Horizontal':
            # In this case, `self.tankDiameter` is the vertical cross section of the horizontal tank.
            hlx_maximumLiquidHeight = (math.pi) * (1. / 4.) * self.tankDiameter
            hln_minimumLiquidHeight = 0

        elif self.tankOrientation == 'Vertical':
            
            hlx_maximumLiquidHeight = self.shellHeight - 1
            hln_minimumLiquidHeight = 1

        else:
            raise ValueError('Assumption Invalid')
        
        # ========================================================================= #      
         
        # n_numberTurnovers
        n_numberTurnovers = hqi_annualSumIncreases /\
                            (hlx_maximumLiquidHeight - hln_minimumLiquidHeight)
        
        # ========================================================================= #
        
        # Assume no splash loading.
        # TODO: Address tanks where flashing occurs.
        # kn_workingLossTurnOverFactor
        if self.isTankOnVaporBalance == True:
            kn_workingLossTurnOverFactor = 1
        
        elif  self.isTankOnVaporBalance == False:
            if n_numberTurnovers <= 36:
                kn_workingLossTurnOverFactor = 1

            elif n_numberTurnovers > 36:
                kn_workingLossTurnOverFactor = (180 + n_numberTurnovers) /\
                                                (6 * n_numberTurnovers)
                
            else:
                raise ValueError('Assumption Invalid')
        else:
            raise ValueError('Assumption Invalid')
            
        # ========================================================================= #
        
        # kp_workingLossProductFactor
        # Assume no splash loading into tank.
        if _productMix[0]['productClass'] in ['Organic Liquids']:
            kp_workingLossProductFactor = 1

        elif _productMix[0]['productClass'] in ['Crude Oils', 'Petroleum Distillates']:
            kp_workingLossProductFactor = 0.75

        else:
            raise ValueError('Assumption Invalid')
             
        # ========================================================================= #    
            
        # vq_netWorkingLossThroughput
        # Use Equation 1-39, check for assumption.
        if self.tankOrientation == 'Horizontal':
                
            effectiveDiameter = ( (self.tankLength * self.tankDiameter) / \
                                 ( (1. / 4.) * (math.pi) ) ) ** (0.5)
            
            _tankDiameter = effectiveDiameter
            
            vq_netWorkingLossThroughput = hqi_annualSumIncreases * (math.pi/4.)*(_tankDiameter)**2
        
        elif self.tankOrientation == 'Vertical':
            
            _tankDiameter = self.tankDiameter
            
            vq_netWorkingLossThroughput = hqi_annualSumIncreases * (math.pi/4.)*((_tankDiameter)**2)

        else:
            raise ValueError('Assumption Invalid')
    
        # ========================================================================= #
   
        pva_vaporPressureAverageDaily = self.vaporSpaceExpansionFactor()['elements'][0]['pva']

        # TODO: Look at Custom Case more carefully. 
        if self.breatherVentSetting == 'Custom':
            
            pbp_breatherVentPressureSetting = self.breatherPressureSetting
            
            _condition =    kn_workingLossTurnOverFactor * \
                            ((pbp_breatherVentPressureSetting + pa_atmosphericPressure) / \
                            (pi_pressureVaporSpace + pa_atmosphericPressure))
            
            if _condition > 1:
                
                kb_ventSettingCorrection = (((pi_pressureVaporSpace + pa_atmosphericPressure) / \
                             kn_workingLossTurnOverFactor) - pva_vaporPressureAverageDaily) / \
                            (pbp_breatherVentPressureSetting + pa_atmosphericPressure - \
                             pva_vaporPressureAverageDaily)
            
            elif _condition <= 1:
                kb_ventSettingCorrection=  kb_ventSettingCorrection
            
            else:
                raise ValueError('Assumption')
            
        # Equation 1-41
        elif self.breatherVentSetting == 'Default':
            kb_ventSettingCorrection = 1 #self.breatherPressureSetting[0] 
        
        else:
            raise ValueError('Assumption Invalid')
        
        workingLoss =   vq_netWorkingLossThroughput *\
                        kn_workingLossTurnOverFactor *\
                        kp_workingLossProductFactor *\
                        self.stockVaporDensity() *\
                        kb_ventSettingCorrection
        
        return {'quantity': 'workingLoss', 
                'value': round(workingLoss, 4),
                'equation': '1-2',
                'version': '06/2020',
                'elements': [{'tankOrientation': self.tankOrientation,
                              'vent setting': self.breatherVentSetting,
                              'pva': pva_vaporPressureAverageDaily,
                             'vq': vq_netWorkingLossThroughput, 
                             'kn': kn_workingLossTurnOverFactor, 
                             'kp': kp_workingLossProductFactor,
                             'wv': self.stockVaporDensity(), 
                             'kb': kb_ventSettingCorrection, 
                              'hlx': hlx_maximumLiquidHeight, 
                              'hln': hln_minimumLiquidHeight, 
                              'q_gal': self.throughput, 
                              'q_bbl': self.throughput / 42,
                              'diameter': _tankDiameter,
                              'num_turnovers': n_numberTurnovers, 
                              'sum_hqi': hqi_annualSumIncreases,
                              'kbv': kb_ventSettingCorrection,
                              'kbp': self.breatherVaccumSetting,
                              'on_vapor_bal':  self.isTankOnVaporBalance,
                              'breatherPressureSetting': self.breatherPressureSetting, 
                              'breatherVaccumSetting': self.breatherVaccumSetting,
                              'roof insulated': self.isRoofInsulated,
                              'shell insulated': self.isShellInsulated,
                              'status': 'Done'}]}
    
    
    
    
    def totalLosses(self):
        '''
        Done -- check.
        Equation 1.1.
        '''
        
        totalLoss = self.workingLosses()['value'] + self.standingLosses()['value']
        
        return {'quantity': 'totalLosses', 
                'equation': '1-1',
                'version': '06/2020',
                'value': totalLoss, 
                'elements': [{'lw': self.workingLosses()['value'],
                              'ls': self.standingLosses()['value'], 
                              'status': 'Done'}]}
    
    def tankInputs(self):
        
        dataDict = {
         'shellHeight': self.shellHeight,
         'tankLength': self.tankLength,
         'tankDiameter': self.tankDiameter, 
         'tankDomeRoofRadius': self.tankDomeRoofRadius, 
         'isTankDomeRoofRadiusKnown': self.isTankDomeRoofRadiusKnown,
         'isTankOnVaporBalance': self.isTankOnVaporBalance,
         'roofType': self.roofType, 
         'tankOrientation': self.tankOrientation, 
         'throughput': self.throughput,
         'breatherVentSetting': self.breatherVentSetting,
         'breatherPressureSetting': self.breatherPressureSetting,
         'breatherVaccumSetting': self.breatherVaccumSetting,
         'location': self.location, 
         'shellShade': self.shellShade, 
         'shellCondition': self.shellCondition, 
         'roofShade': self.roofShade, 
         'roofCondition': self.roofCondition, 
         'isShellInsulated': self.isShellInsulated,
         'isRoofInsulated': self.isRoofInsulated, 
         'productClassString': self.productMix[0]['productClass'], 
         'productNameString': self.productMix[0]['productName'], 
         'roofSlope': self.roofSlope, 
         'isAverageLiquidHeightKnown': self.isAverageLiquidHeightKnown,
         'averageLiquidHeight': self.averageLiquidHeight, 
         'tankName': self.tankName,
         'assetNumber': self.assetNumber}

        return dataDict


In [8]:
#productMix = [{'productClass': 'Crude Oils', 'productName': 'Crude oil (RVP 5)', 'composition': 1}]  
#productMix = [{'productClass': 'Petroleum Distillates', 'productName': 'Gasoline (RVP 8)', 'composition': 1}]  
productMix = [{'productClass': 'Organic Liquids', 'productName': 'Benzene', 'composition': 2812/3171}, 
              {'productClass': 'Organic Liquids', 'productName': 'Toluene', 'composition': 258/3171}, 
              {'productClass': 'Organic Liquids', 'productName': 'Cyclohexane', 'composition': 101/3171}]
    
obj = FixedRoofTank(
              location='Denver, CO',
              shellShade='White', 
              shellCondition='Average',
              roofShade='White',
              roofCondition='Average',
              productMix=productMix, 
              isShellInsulated=False, 
              isRoofInsulated=False, 
              shellHeight=12,
              tankDiameter=6, 
              tankOrientation ='Vertical', 
              isAverageLiquidHeightKnown=False, 
              tankLength=20, 
              roofType='Cone',
              tankDomeRoofRadius=10, 
              roofSlope=0.0625,                  
              breatherPressureSetting=0.03, 
              breatherVaccumSetting=-0.03, 
              throughput=8450, 
              isTankOnVaporBalance=False, 
              breatherVentSetting='Default',          
              tankName=None,
              assetNumber=None, 
              isTankDomeRoofRadiusKnown=True, 
              averageLiquidHeight=True)

a = obj.tankInputs()
b = obj.totalLosses()
c = obj.resultsShellRoofData() 
d = obj.SQLResultsLocation()
e = obj.SQLResultsProductData()
f = obj.locationData()
g = obj.temperatureCalculations()
h = obj.organicLiquidMixtureVaporMolecularWeight()
#i = obj.crudePetVaporMolecularWeight()
j = obj.stockVaporDensity()
k = obj.vaporSpaceOutage()
l = obj.vaporSpaceVolume()
m = obj.workingLosses()
n = obj.standingLosses()
o = obj.vaporSpaceExpansionFactor()
p = obj.ventedVaporSpaceSatFactor()

In [10]:
#[each for each in [a, b, c, d, e, f, g, i, j, k, l, m, n, o, p]]

In [ ]:
import json
category1 = 'Organic Liquids'
datain1 = [{'name': 'Acrylonitrile', 'composition': 0.30}, 
           {'name': 'Acetone', 'composition': 0.60}, 
           {'name': 'Acetic anhydride', 'composition': 0.10}]

category2 = 'Crude Oils'
datain2 = 'Crude oil (RVP 5)'

def returnData1(productClassString=None,productNameString=None):
        
    obj =   VerticalFixedRoofTank(
            shellHeight=20, 
            tankDiameter=15, 
            tankDomeRoofRadius=9, 
            breatherPressureSetting=0.03,
            breatherVaccumSetting=-0.03,
            tankLength=10,
            throughput=10000, 
            roofSlope=0.0625, 
            averageLiquidHeight=10/2, 
            rvp_crudeOils=5,
            productMW=50,
            isTankDomeRoofRadiusKnown=False, 
            isTankOnVaporBalance=False, 
            isRoofInsulated=False,
            isShellInsulated=False,   
            isAverageLiquidHeightKnown=False,
            isCustomProduct=False,
            roofType='Cone',
            tankOrientation='Vertical', 
            productType='Crude Oils', 
            breatherVentSetting='Default',
            location='Long Beach, CA',
            shellShade='White', 
            shellCondition='Average', 
            roofShade='White',
            roofCondition='Average', 
            productClassString=productClassString,
            productNameString=productNameString,         
            tankName='North Gas Oil Tank',
            assetNumber='4209')

    a = obj.averageAmbientTempData()
    b = obj.vaporSpaceVolume()
    c = obj.ventedVaporSpaceSatFactor()
    d = obj.vaporSpaceOutage()
    e = obj.stockVaporDensity()
    f = obj.vaporSpaceExpansionFactor()
    g = obj.totalLosses()
    h = obj.standingLosses()
    i = obj.workingLosses()
    j = obj.tankInputs()

    data = [ {'index': 0, 'method': 'averageAmbientTempData', 'data': a}, 
             {'index': 1, 'method': 'vaporSpaceVolume', 'data': b}, 
             {'index': 2, 'method': 'ventedVaporSpaceSatFactor', 'data': c}, 
             {'index': 3, 'method': 'vaporSpaceOutage', 'data': d},
             {'index': 4, 'method': 'stockVaporDensity', 'data': e}, 
             {'index': 5, 'method': 'vaporSpaceExpansionFactor', 'data': f}, 
             {'index': 6, 'method': 'totalLosses', 'data': g}, 
             {'index': 7, 'method': 'standingLosses', 'data': h},
             {'index': 8, 'method': 'workingLosses', 'data': i}, 
             {'index': 9, 'method': 'tankInputs', 'data': j}]
    
    #tank dimensions
    hs = data[9]['data']['shellHeight']
    d = data[8]['data']['elements'][0]['diameter']
    hlx = data[8]['data']['elements'][0]['hlx']
    hln = data[8]['data']['elements'][0]['hln']
    hl = data[3]['data']['elements'][0]['hl'] #ok
    n = data[8]['data']['elements'][0]['num_turnovers']
    throughput_gal = data[8]['data']['elements'][0]['q_gal']
    q_bbl = data[8]['data']['elements'][0]['q_bbl']
    on_vapor_bal = data[8]['data']['elements'][0]['on_vapor_bal']
    
    #paint characteristics
    shell_color = data[0]['data']['shell_shade']
    shell_cond = data[0]['data']['shell_condition']
    roof_color = data[0]['data']['roof_shade']
    roof_cond = data[0]['data']['roof_condition']
    
    #roof characteristics
    roof_type = data[9]['data']['roofType']
    hr = data[3]['data']['elements'][0]['hr']
    slope = data[9]['data']['roofSlope']
    
    #Breather Vent
    pbv = data[8]['data']['elements'][0]['breatherVaccumSetting']
    pbp = data[8]['data']['elements'][0]['breatherPressureSetting']
    
    #Insulation
    is_shell_insulated = data[8]['data']['elements'][0]['shell insulated']
    is_roof_insulated = data[8]['data']['elements'][0]['roof insulated']
    
    #MET Data
    city = data[0]['data']['city']
    taa = data[0]['data']['taa_averageDailyTemp']
    tax = data[0]['data']['tax_maxAmbientTemp_R'] 
    tan = data[0]['data']['tan_minAmbientTemp_R']
    del_ta = data[0]['data']['delta_ta_R']
    v =  data[0]['data']['wind_speed'] 
    i = data[0]['data']['i_solarInsulation'] 
    pa = data[0]['data']['atmPressure'] 
    
    #Liquid Data
    liq_category = data[5]['data']['elements'][0]['product class']
    liq_name = data[5]['data']['elements'][0]['product name']
    tb = data[0]['data']['tb_liquidBulkTemp']
    tla = data[5]['data']['elements'][0]['tla']
    tln = data[5]['data']['elements'][0]['tln']
    tlx = data[5]['data']['elements'][0]['tlx']
    pva = data[5]['data']['elements'][0]['pva']
    pvn = data[5]['data']['elements'][0]['pvn']
    pvx = data[5]['data']['elements'][0]['pvx']
    mv = data[4]['data']['elements'][0]['mv']
    rvp = data[5]['data']['elements'][0]['rvp']
    const_a = data[5]['data']['elements'][0]['const_a']
    const_b = data[5]['data']['elements'][0]['const_b']
    
    #Standing Losses
    ls = data[7]['data']['value']
    vv = data[7]['data']['elements'][0]['vv']
    wv= data[7]['data']['elements'][0]['wv']
    ke= data[7]['data']['elements'][0]['ke']
    ks= data[7]['data']['elements'][0]['ks']
    
    #Working Losses
    lw = data[8]['data']['value']
    vq = data[8]['data']['elements'][0]['vq']   
    kn = data[8]['data']['elements'][0]['kn']   
    kp = data[8]['data']['elements'][0]['kp']
    wv = data[8]['data']['elements'][0]['wv']   
    kb = data[8]['data']['elements'][0]['kb']
    
    #Total Losses
    lw = data[6]['data']['elements'][0]['lw'] 
    ls = data[6]['data']['elements'][0]['ls'] 
    lt = data[6]['data']['value']
    
    #Vapor Space Volume
    vv = data[7]['data']['elements'][0]['vv']
    diameter = data[8]['data']['elements'][0]['diameter']  
    hvo = data[1]['data']['elements'][0]['hvo']
    
    #Vapor Space Outage
    hvo = data[1]['data']['elements'][0]['hvo']
    hs = data[9]['data']['shellHeight']
    hl = data[3]['data']['elements'][0]['hl'] #ok
    hro = data[3]['data']['elements'][0]['hro'] #ok
    
    #Roof Outage
    hro = data[3]['data']['elements'][0]['hro'] #ok
    hr = data[3]['data']['elements'][0]['hr']
    rs = data[9]['data']['tankDiameter'] / 2.
    sr = data[9]['data']['roofSlope']
    
    #Vapor Density
    wv = data[8]['data']['elements'][0]['wv']
    mv = data[4]['data']['elements'][0]['mv']
    pva = data[5]['data']['elements'][0]['pva']
    r = data[4]['data']['elements'][0]['constant']
    tv = data[4]['data']['elements'][0]['tv']
    alpha_s = data[0]['data']['alphaS_shell']
    alpha_r = data[0]['data']['alphaR_roof']
    i_value = data[0]['data']['i_solarInsulation']
    
    #Vapor Space Expansion Factor
    ke= data[7]['data']['elements'][0]['ke']
    del_tv = data[5]['data']['elements'][0]['delta_tv']
    delta_pv = data[5]['data']['elements'][0]['delta_pv']
    delta_pb = data[5]['data']['elements'][0]['delta_pb']
    pa = data[5]['data']['elements'][0]['atm_pressure']
    pva = data[5]['data']['elements'][0]['pva']
    tla = data[5]['data']['elements'][0]['tla']

    #Vented Vapor Space Saturation Factor
    ks = data[2]['data']['value']
    pva = data[2]['data']['elements'][0]['pva']
    hvo = data[2]['data']['elements'][0]['hvo']
    
    return_data = {

        'tank_dimensions': [{

            'value': None,
            'parameter_estimates': [{'parameter_name': 'Shell Height, ft', 'parameter_symbol': 'hs', 'parameter_value': hs},
                                    {'parameter_name': 'Shell Diameter, ft', 'parameter_symbol': 'd', 'parameter_value': d},
                                    {'parameter_name': 'Max Liquid Height, ft', 'parameter_symbol': 'hlx', 'parameter_value': hlx},
                                    {'parameter_name': 'Min Liquid Height, ft', 'parameter_symbol': 'hln', 'parameter_value': hln},
                                    {'parameter_name': 'Average Liquid Height, ft', 'parameter_symbol': 'hl', 'parameter_value': hl},
                                    {'parameter_name': 'Number of Turnovers per Year, unitless', 'parameter_symbol': 'n', 'parameter_value': n},
                                    {'parameter_name': 'Net Annual Throughput, gal', 'parameter_symbol': 'q_gal', 'parameter_value': throughput_gal},
                                    {'parameter_name': 'Net Annual Throughput, bbl', 'parameter_symbol': 'q_bbl', 'parameter_value': q_bbl},
                                    {'parameter_name': 'Vapor Balanced, bool', 'parameter_symbol': 'on_vapor_bal','parameter_value': on_vapor_bal}]
            }],

        'paint_characteristics': [{

            'value': None,
            'parameter_estimates': [{'parameter_name': 'Shell Color', 'parameter_symbol': 'shell_color', 'parameter_value': shell_color},
                                    {'parameter_name': 'Shell Condition', 'parameter_symbol': 'shell_cond', 'parameter_value': shell_cond},
                                    {'parameter_name': 'Roof Color', 'parameter_symbol': 'roof_color', 'parameter_value': roof_color},
                                    {'parameter_name': 'Roof Condition', 'parameter_symbol': 'roof_cond', 'parameter_value': roof_cond}]
        }],

        'roof_characteristics': [{

            'value': None,
            'parameter_estimates': [{'parameter_name': 'roof type', 'parameter_symbol': 'roof_type', 'parameter_value': roof_type},
                                    {'parameter_name': 'roof height, ft', 'parameter_symbol': 'hr', 'parameter_value': hr},
                                    {'parameter_name': 'roof slope, ft/ft', 'parameter_symbol': 'slope', 'parameter_value': slope}]
        }],


        'breather_vent_settings': [{

            'value': None,
            'parameter_estimates': [{'parameter_name': 'Breather Vent Vacuum Setting', 'parameter_symbol': 'pbv, psig', 'parameter_value': pbv[0]},
                                    {'parameter_name': 'Breather Vent Pressure Setting', 'parameter_symbol': 'pbp, psig', 'parameter_value': pbp[0]}]
        }],


        'insulation_characteristics': [{

            'value': None,
            'parameter_estimates': [{'parameter_name': 'Shell Insulation', 'parameter_symbol': 'is_shell_insulated, bool', 'parameter_value': is_shell_insulated},
                                    {'parameter_name': 'Roof Insulation', 'parameter_symbol': 'is_roof_insulated, bool', 'parameter_value': is_roof_insulated}]
        }],

        'meterological_data': [{

            'value': None,
            'parameter_estimates': [{'parameter_name': 'City', 'parameter_symbol': 'city', 'parameter_value': city},
                                    {'parameter_name': 'Average Daily Ambient Temp', 'parameter_symbol': 'taa, R', 'parameter_value': taa},
                                    {'parameter_name': 'Average Daily Max Ambient Temp', 'parameter_symbol': 'tax, R', 'parameter_value': tax},
                                    {'parameter_name': 'Average Daily Min Ambient Temp', 'parameter_symbol': 'tan, R', 'parameter_value': tan},
                                    {'parameter_name': 'Average Daily Ambient Temp Range', 'parameter_symbol': 'del_ta, R', 'parameter_value': del_ta},
                                    {'parameter_name': 'Average Wind Speed', 'parameter_symbol': 'v, miles/hr', 'parameter_value': v},
                                    {'parameter_name': 'Average Daily Total Insolation Factor', 'parameter_symbol': 'i, btu/ft^2-day', 'parameter_value': i},
                                    {'parameter_name': 'Atm. Pressure', 'parameter_symbol': 'pa, psi', 'parameter_value': pa}]
        }],

        'liquid_data': [{

            'value': None,
            'parameter_estimates': [{'parameter_name': 'Liquid Category', 'parameter_symbol': 'liq_category', 'parameter_value': liq_category},
                                    {'parameter_name': 'Liquid Name', 'parameter_symbol': 'liq_name', 'parameter_value': liq_name},
                                    {'parameter_name': 'Liquid Bulk Temp', 'parameter_symbol': 'tb', 'parameter_value': tb},
                                    {'parameter_name': 'Average Daily Liquid Surface Temp, R', 'parameter_symbol': 'tla', 'parameter_value': tla},
                                    {'parameter_name': 'Average Daily Min Liquid Surface Temp, R', 'parameter_symbol': 'tln', 'parameter_value': tln},
                                    {'parameter_name': 'Average Daily Max Liquid Surface Temp, R', 'parameter_symbol': 'tlx', 'parameter_value': tlx},
                                    {'parameter_name': 'Vapor Pressure at the Ave. Daily Min Liquid Surface Temp, R', 'parameter_symbol': 'pva', 'parameter_value': pva},
                                    {'parameter_name': 'Vapor Pressure at the Ave. Daily Max Liquid Surface Temp, R', 'parameter_symbol': 'pvn', 'parameter_value': pvn},
                                    {'parameter_name': 'Vapor Molecular Weight lb/lb-mole', 'parameter_symbol': 'pvx', 'parameter_value': pvx},
                                    {'parameter_name': 'RVP, psia', 'parameter_symbol': 'mv', 'parameter_value': mv},
                                    {'parameter_name': 'Grant', 'parameter_symbol': 'rvp', 'parameter_value': rvp},
                                    {'parameter_name': 'Constant A, unitless', 'parameter_symbol': 'const_a', 'parameter_value': const_a},
                                    {'parameter_name': 'Constant B, R', 'parameter_symbol': 'const_b', 'parameter_value': const_b}]
        }],

        'standing_losses': [{

            'value': ls,
            'parameter_estimates': [{'parameter_name': 'Standing Losses, lbs', 'parameter_symbol': 'ls', 'parameter_value': ls},
                                    {'parameter_name': 'Vapor Space Volume, ft^3', 'parameter_symbol': 'vv', 'parameter_value': vv},
                                    {'parameter_name': 'Vapor Density, lb/ft^3', 'parameter_symbol': 'wv', 'parameter_value': wv},
                                    {'parameter_name': 'Vapor Space Expansion Factor, 1/day', 'parameter_symbol': 'ke', 'parameter_value': ke},
                                    {'parameter_name': 'Vented Vapor Saturation Factor, unitless', 'parameter_symbol': 'ks', 'parameter_value': ks}]
        }],

        'working_losses': [{

            'value': lw,
            'parameter_estimates': [{'parameter_name': 'Working Losses, lbs/year', 'parameter_symbol': 'lw', 'parameter_value': lw},
                                    {'parameter_name': 'Net Working Loss Throughput, ft^3/year', 'parameter_symbol': 'vq', 'parameter_value': vq},
                                    {'parameter_name': 'Turnover Factor, unitless', 'parameter_symbol': 'kn', 'parameter_value': kn},
                                    {'parameter_name': 'Working Loss Product Factor, unitless', 'parameter_symbol': 'kp', 'parameter_value': kp},
                                    {'parameter_name': 'Vapor Density, lb/ft^3', 'parameter_symbol': 'wv', 'parameter_value': wv},
                                    {'parameter_name': 'Vent Setting Correction Factor, unitless', 'parameter_symbol': 'kb', 'parameter_value': kb}]
        }],

        'total_losses': [{

            'value': lt,
            'parameter_estimates': [{'parameter_name': 'working_loss, lbs', 'parameter_symbol': 'lw', 'parameter_value': lw},
                                    {'parameter_name': 'standing_loss, lbs', 'parameter_symbol': 'ls', 'parameter_value': ls},
                                    {'parameter_name': 'total_loss, lbs', 'parameter_symbol': 'lt', 'parameter_value': lt}]
        }],

        'vapor_space_volume': [{

            'value': vv,
            'parameter_estimates': [{'parameter_name': 'Vapor Space Volume', 'parameter_symbol': 'vv', 'parameter_value': vv},
                                    {'parameter_name': 'Tank Diameter', 'parameter_symbol': 'diameter', 'parameter_value': diameter},
                                    {'parameter_name': 'Vapor Space Outage', 'parameter_symbol': 'hvo', 'parameter_value': hvo}]
        }],

        'vapor_space_outage': [{

            'value': hvo,
            'parameter_estimates': [{'parameter_name': 'Vapor Space Outage', 'parameter_symbol': 'hvo', 'parameter_value': hvo},
                                    {'parameter_name': 'Tank Shell Height', 'parameter_symbol': 'hs', 'parameter_value': hs},
                                    {'parameter_name': 'Liquid Height', 'parameter_symbol': 'hl', 'parameter_value': hl},
                                    {'parameter_name': 'Roof Outage', 'parameter_symbol': 'hro', 'parameter_value': hro}]
        }],

        'roof_outage': [{

            'value': hro,
            'parameter_estimates': [{'parameter_name': 'Roof Outage', 'parameter_symbol': 'hro', 'parameter_value': hro},
                                    {'parameter_name': 'Tank Roof Height', 'parameter_symbol': 'hr', 'parameter_value': hr},
                                    {'parameter_name': 'Tank Shell Radius', 'parameter_symbol': 'rs', 'parameter_value': rs},
                                    {'parameter_name': 'Tank Cone Roof Slope', 'parameter_symbol': 'sr', 'parameter_value': sr}]
        }],

        'vapor_density': [{

            'value': wv,
            'parameter_estimates': [{'parameter_name': 'Vapor Density, lb/ft^3', 'parameter_symbol': 'wv', 'parameter_value': wv},
                                    {'parameter_name': 'Vapor Molecular Weight, lb/lb-mole', 'parameter_symbol': 'mv', 'parameter_value': mv},
                                    {'parameter_name': 'Vapor Pressure at Average Daily Liquid Surface Temp, psia', 'parameter_symbol': 'pva', 'parameter_value': pva},
                                    {'parameter_name': 'Ideal Gas Constant', 'parameter_symbol': 'r', 'parameter_value': r},
                                    {'parameter_name': 'Average Vapor Temperature', 'parameter_symbol': 'tv', 'parameter_value': tv},
                                    {'parameter_name': 'Tank Roof Surface Solar Absorptance', 'parameter_symbol': 'alpha_s', 'parameter_value': alpha_s},
                                    {'parameter_name': 'Tank Shell Surface Solar Absorptance', 'parameter_symbol': 'alpha_r', 'parameter_value': alpha_r},
                                    {'parameter_name': 'Average Daily Total Insolation Factor', 'parameter_symbol': 'i_value', 'parameter_value': i_value}]
        }],

        'vapor_space_expansion_factor': [{

            'value': ke,
            'parameter_estimates': [{'parameter_name': 'Vapor Space Expansion Factor, 1/day', 'parameter_symbol': 'ke', 'parameter_value': ke},
                                    {'parameter_name': 'Average Daily Vapor Presure Temp. Range, R', 'parameter_symbol': 'del_tv', 'parameter_value': del_tv},
                                    {'parameter_name': 'Average Daily Vapor Pressure Range, psi', 'parameter_symbol': 'delta_pv', 'parameter_value': delta_pv},
                                    {'parameter_name': 'Breather Vent Pressure Setting Range, psig', 'parameter_symbol': 'delta_pb', 'parameter_value': delta_pb},
                                    {'parameter_name': 'Atmospheric Pressure, psi', 'parameter_symbol': 'pa', 'parameter_value': pa},
                                    {'parameter_name': 'Vapor Pressure at Average Daily Liquid Surface Temp, psia', 'parameter_symbol': 'pva', 'parameter_value': pva},
                                    {'parameter_name': 'Average Daily Liquid Surface Temp, R', 'parameter_symbol': 'tla', 'parameter_value': tla}]
        }],

        'vented_vapor_space_saturation_factor': [{

            'value': ks,
            'parameter_estimates': [{'parameter_name': 'Vented Vapor Saturation Factor, unitless', 'parameter_symbol': 'ks', 'parameter_value': ks},
                                    {'parameter_name': 'Vapor Pressure at Average Daily Liquid Surface Temp, psia', 'parameter_symbol': 'pva', 'parameter_value': pva},
                                    {'parameter_name': 'Vapor Space Outage, ft', 'parameter_symbol': 'hvo', 'parameter_value': hvo}]
        }],
        
        'data_inputs': [{
            
            'value': None,
            'parameter_estimates': [{'parameter_name': 'hs_shellHeight', 'parameter_symbol': 'g', 'value': data[9]['data']['shellHeight']}, 
                                    {'parameter_name': 'tankLength', 'parameter_symbol': 'g', 'value': data[9]['data']['tankLength']},
                                    {'parameter_name': 'tankDiameter', 'parameter_symbol': 'g', 'value': data[9]['data']['tankDiameter']},
                                    {'parameter_name': 'rr_tankDomeRoofRadius', 'parameter_symbol': 'g', 'value': data[9]['data']['tankDomeRoofRadius']},
                                    {'parameter_name': 'isTankDomeRoofRadiusKnown', 'parameter_symbol': 'g', 'value': data[9]['data']['isTankDomeRoofRadiusKnown']},
                                    {'parameter_name': 'isTankOnVaporBalance', 'parameter_symbol': 'g', 'value': data[9]['data']['isTankOnVaporBalance']},
                                    {'parameter_name': 'roofType', 'parameter_symbol': 'g', 'value': data[9]['data']['roofType']},
                                    {'parameter_name': 'tankOrientation', 'parameter_symbol': 'g', 'value': data[9]['data']['tankOrientation']},
                                    {'parameter_name': 'throughput', 'parameter_symbol': 'g', 'value': data[9]['data']['throughput']},
                                    {'parameter_name': 'productType', 'parameter_symbol': 'g', 'value': data[9]['data']['productType']},
                                    {'parameter_name': 'breatherVentSetting', 'parameter_symbol': 'g', 'value': data[9]['data']['breatherVentSetting']},
                                    {'parameter_name': 'breatherPressureSetting', 'parameter_symbol': 'g', 'value': data[9]['data']['breatherPressureSetting']},
                                    {'parameter_name': 'breatherVaccumSetting', 'parameter_symbol': 'g', 'value': data[9]['data']['breatherVaccumSetting']},
                                    {'parameter_name': 'location', 'parameter_symbol': 'g', 'value': data[9]['data']['location']},
                                    {'parameter_name': 'shellShade', 'parameter_symbol': 'g', 'value': data[9]['data']['shellShade']},
                                    {'parameter_name': 'shellCondition', 'parameter_symbol': 'g', 'value': data[9]['data']['shellCondition']},
                                    {'parameter_name': 'roofShade', 'parameter_symbol': 'g', 'value': data[9]['data']['roofShade']},
                                    {'parameter_name': 'roofCondition', 'parameter_symbol': 'g', 'value': data[9]['data']['roofCondition']},
                                    {'parameter_name': 'isShellInsulated', 'parameter_symbol': 'g', 'value': data[9]['data']['isShellInsulated']},
                                    {'parameter_name': 'isRoofInsulated', 'parameter_symbol': 'g', 'value': data[9]['data']['isRoofInsulated']},
                                    {'parameter_name': 'productClassString', 'parameter_symbol': 'g', 'value': data[9]['data']['productClassString']},
                                    {'parameter_name': 'productNameString', 'parameter_symbol': 'g', 'value': data[9]['data']['productNameString']},
                                    {'parameter_name': 'roofSlope', 'parameter_symbol': 'g', 'value': data[9]['data']['roofSlope']},
                                    {'parameter_name': 'isAverageLiquidHeightKnown', 'parameter_symbol': 'g', 'value': data[9]['data']['isAverageLiquidHeightKnown']},
                                    {'parameter_name': 'averageLiquidHeight', 'parameter_symbol': 'g', 'value': data[9]['data']['averageLiquidHeight']}, 
                                    {'parameter_name': 'rvp_crudeOils', 'parameter_symbol': 'g', 'value': data[9]['data']['rvp_crudeOils']},
                                    {'parameter_name': 'tankName', 'parameter_symbol': 'g', 'value': data[9]['data']['tankName']}, 
                                    {'parameter_name': 'assetNumber', 'parameter_symbol': 'g', 'value': data[9]['data']['assetNumber']}]
        }]

    }
    
    return json.dumps(return_data)

In [ ]:
category2 = 'Crude Oils'
datain2 = 'Crude oil (RVP 5)'
d = returnData1(productClassString=category2,productNameString=datain2)

In [ ]:
category1 = 'Organic Liquids'
datain1 = [{'name': 'Acrylonitrile', 'composition': 0.30}, 
           {'name': 'Acetone', 'composition': 0.60}, 
           {'name': 'Acetic anhydride', 'composition': 0.10}]

category2 = 'Crude Oils'
datain2 = 'Crude oil (RVP 5)'
        
obj =   VerticalFixedRoofTank(
            shellHeight=20, 
            tankDiameter=10, 
            tankDomeRoofRadius=9, 
            breatherPressureSetting=0.03,
            breatherVaccumSetting=-0.03,
            tankLength=15,
            throughput=9691.1600, 
            roofSlope=0.0625, 
            averageLiquidHeight=10/2, 
            rvp_crudeOils=5,
            productMW=50,
            isTankDomeRoofRadiusKnown=False, 
            isTankOnVaporBalance=False, 
            isRoofInsulated=False,
            isShellInsulated=False,   
            isAverageLiquidHeightKnown=False,
            isCustomProduct=False,
            roofType='Cone',
            tankOrientation='Horizontal', 
            productType='Crude Oils', 
            breatherVentSetting='Default',
            location='Long Beach, CA',
            shellShade='White', 
            shellCondition='Average', 
            roofShade='White',
            roofCondition='Average', 
            productClassString=category2,
            productNameString=datain2,         
            tankName='North Gas Oil Tank',
            assetNumber='4209')

a = obj.averageAmbientTempData()
b = obj.vaporSpaceVolume()
c = obj.ventedVaporSpaceSatFactor()
d = obj.vaporSpaceOutage()
e = obj.stockVaporDensity()
f = obj.vaporSpaceExpansionFactor()
g = obj.totalLosses()
h = obj.standingLosses()
i = obj.workingLosses()
j = obj.tankInputs()

data = [ {'index': 0, 'method': 'averageAmbientTempData', 'data': a}, 
         {'index': 1, 'method': 'vaporSpaceVolume', 'data': b}, 
         {'index': 2, 'method': 'ventedVaporSpaceSatFactor', 'data': c}, 
         {'index': 3, 'method': 'vaporSpaceOutage', 'data': d},
         {'index': 4, 'method': 'stockVaporDensity', 'data': e}, 
         {'index': 5, 'method': 'vaporSpaceExpansionFactor', 'data': f}, 
         {'index': 6, 'method': 'totalLosses', 'data': g}, 
         {'index': 7, 'method': 'standingLosses', 'data': h},
         {'index': 8, 'method': 'workingLosses', 'data': i}, 
         {'index': 9, 'method': 'tankInputs', 'data': j}]

### Tank Dimensions

In [ ]:
hs = data[3]['data']['elements'][0]['shell height']
d = data[8]['data']['elements'][0]['diameter']
hlx = data[8]['data']['elements'][0]['hlx']
hln = data[8]['data']['elements'][0]['hln']
hl = data[3]['data']['elements'][0]['hl']
n = data[8]['data']['elements'][0]['num_turnovers']
throughput_gal = data[8]['data']['elements'][0]['q_gal']
q_bbl = data[8]['data']['elements'][0]['q_bbl']
on_vapor_bal = data[8]['data']['elements'][0]['on_vapor_bal']

print('-----------------------')
print('Tank Dimensions')
print('hs:{} // d:{} // hlx:{} // hln:{} // hl:{} // n:{} // throughput_gal:{} // q_bbl:{} // on_vapor_bal:{}'.format(hs, d, hlx, hln, hl, n, throughput_gal, q_bbl, on_vapor_bal))

df_tank_dimensions = pd.DataFrame({'quantity': ['Tank Dimensions' for each in range(9)], 
                                   'parameter_name': ['Shell Height, ft', 
                                                      'Shell Diameter, ft', 
                                                      'Max Liquid Height, ft', 
                                                      'Min Liquid Height, ft', 
                                                      'Average Liquid Height, ft', 
                                                      'Number of Turnovers per Year, unitless', 
                                                      'Net Annual Throughput, gal', 
                                                      'Net Annual Throughput, bbl', 
                                                      'Vapor Balanced, bool'], 
                                   'parameter': ['hs', 
                                                 'd', 
                                                 'hlx', 
                                                 'hln', 
                                                 'hl', 
                                                 'n', 
                                                 'q_gal', 
                                                 'q_bbl', 
                                                 'on_vapor_bal'], 
                                   'value': [hs, 
                                             d, 
                                             hlx, 
                                             hln, 
                                             hl, 
                                             n, 
                                             throughput_gal, 
                                             q_bbl, 
                                             on_vapor_bal]})


### Paint Characteristics

In [ ]:
shell_color = data[0]['data']['shell_shade']
shell_cond = data[0]['data']['shell_condition']
roof_color = data[0]['data']['roof_shade']
roof_cond = data[0]['data']['roof_condition']
print('-----------------------')
print('Paint Characteristics')
print('shell_color: {} // shell_cond: {} // roof_color: {} // roof_cond: {} //'.format(shell_color, shell_cond, roof_color, roof_cond))


df_paint_characteristics = pd.DataFrame({'quantity': ['Paint Characteristics' for each in range(4)], 
                                         'parameter_name': ['Shell Color', 
                                                            'Shell Condition', 
                                                            'Roof Color', 
                                                            'Roof Condition'], 
                                         'parameter': ['shell_color', 
                                                       'shell_cond', 
                                                       'roof_color', 
                                                       'roof_cond'], 
                                         'value': [shell_color, 
                                                   shell_cond, 
                                                   roof_color, 
                                                   roof_cond]})


### Roof Characteristics

In [ ]:
roof_type = data[3]['data']['elements'][0]['roof type']
hr = data[3]['data']['elements'][0]['hr']
slope = data[3]['data']['elements'][0]['roof slope']
print('-----------------------')
print('Roof Characteristics')
print('roof_type: {} // hr: {} // slope: {} //'.format(roof_type, hr, slope))

df_roof_characteristics = pd.DataFrame({'quantity': ['Roof Characteristics', 
                                                     'Roof Characteristics', 
                                                     'Roof Characteristics'], 
                                        'parameter_name': ['roof type', 
                                                           'roof height, ft', 
                                                           'roof slope, ft/ft'], 
                                        'parameter': ['roof_type', 
                                                      'hr', 
                                                      'slope'], 
                                        'value': [roof_type, 
                                                  hr, 
                                                  slope]})

### Breather Vent Settings

In [ ]:
pbv = data[8]['data']['elements'][0]['breatherVaccumSetting']
pbp = data[8]['data']['elements'][0]['breatherPressureSetting']
print('-----------------------')
print('Breather Vent Settings')
print('pbv: {} // pbp: {}'.format(pbv, pbp))

df_breather_vent_settings = pd.DataFrame({'quantity': ['Breather Vent Settings', 
                                                       'Breather Vent Settings'], 
                                          'parameter_name': ['Breather Vent Vacuum Setting', 
                                                             'Breather Vent Pressure Setting'], 
                                          'parameter': ['pbv, psig', 
                                                        'pbp, psig'], 
                                          'value': [pbv[0], 
                                                    pbp[0]]})

### Insulation Characteristics

In [ ]:
is_shell_insulated = data[8]['data']['elements'][0]['shell insulated']
is_roof_insulated = data[8]['data']['elements'][0]['roof insulated']
print('-----------------------')
print('Insolation Characteristics')
print('is_shell_insulated: {} // is_roof_insulated: {}'.format(is_shell_insulated, is_roof_insulated))

df_insulation_characteristics = pd.DataFrame({'quantity': ['Insulation Characteristics', 
                                                           'Insulation Characteristics'], 
                                              'parameter_name': ['Shell Insulation', 
                                                                 'Roof Insulation'], 
                                              'parameter': ['is_shell_insulated, bool', 
                                                            'is_roof_insulated, bool'], 
                                              'value': [is_shell_insulated, 
                                                        is_roof_insulated]})

### Meterological Data

In [ ]:
city = data[0]['data']['city']
taa = data[0]['data']['taa_averageDailyTemp']
tax = data[0]['data']['tax_maxAmbientTemp_R'] 
tan = data[0]['data']['tan_minAmbientTemp_R']
del_ta = data[0]['data']['delta_ta_R']
v =  data[0]['data']['wind_speed'] 
i = data[0]['data']['i_solarInsulation'] 
pa = data[0]['data']['atmPressure'] 
print('-----------------------')
print('Meterological Data')
print('city: {} // taa: {} // tax: {} // tan: {} // del_ta: {} // v: {} // i: {} // pa: {} //'.format(city, taa, tax, tan, del_ta, v, i, pa))

df_meterological_data = pd.DataFrame({'quantity': ['Meterological Data' for each in range(8)], 
                                      'parameter_name': ['City', 
                                                         'Average Daily Ambient Temp', 
                                                         'Average Daily Max Ambient Temp', 
                                                         'Average Daily Min Ambient Temp', 
                                                         'Average Daily Ambient Temp Range', 
                                                         'Average Wind Speed', 
                                                         'Average Daily Total Insolation Factor', 
                                                         'Atm. Pressure'], 
                                      'parameter': ['city', 
                                                    'taa, R', 
                                                    'tax, R', 
                                                    'tan, R', 
                                                    'del_ta, R', 
                                                    'v, miles/hr', 
                                                    'i, btu/ft^2-day', 
                                                    'pa, psi'], 
                                      'value': [city, 
                                                taa, 
                                                tax, 
                                                tan, 
                                                del_ta, 
                                                v, 
                                                i, 
                                                pa]})


### Liquid Data

In [ ]:
liq_category = data[5]['data']['elements'][0]['product class']
liq_name = data[5]['data']['elements'][0]['product name']
tb = data[0]['data']['tb_liquidBulkTemp']
tla = data[5]['data']['elements'][0]['tla']
tln = data[5]['data']['elements'][0]['tln']
tlx = data[5]['data']['elements'][0]['tlx']
pva = data[5]['data']['elements'][0]['pva']
pvn = data[5]['data']['elements'][0]['pvn']
pvx = data[5]['data']['elements'][0]['pvx']
mv = data[4]['data']['elements'][0]['mv']
rvp = data[5]['data']['elements'][0]['rvp']
const_a = data[5]['data']['elements'][0]['const_a']
const_b = data[5]['data']['elements'][0]['const_b']

print('-----------------------')
print('Liquid Data')
print('liq_category: {} // liq_name: {} // tb: {} // tla: {} // tln: {} // tlx: {} // pva: {} //'.format(liq_category, liq_name, tb, tla, tln, tlx, pva))
print('---')
print('pvn: {} // pvx: {} //mv: {} // rvp: {} // const_a: {} // const_b: {} //'.format(pvn, pvx, mv, rvp, const_a, const_b))



df_liquid_data = pd.DataFrame({'quantity': ['Liquid Data' for each in range(13)], 
              'parameter_name': ['Liquid Category', 
                                    'Liquid Name', 
                                    'Liquid Bulk Temp', 
                                    'Average Daily Liquid Surface Temp, R', 
                                    'Average Daily Min Liquid Surface Temp, R', 
                                    'Average Daily Max Liquid Surface Temp, R', 
                                    'Vapor Pressure at the Ave. Daily Min Liquid Surface Temp, R', 
                                    'Vapor Pressure at the Ave. Daily Max Liquid Surface Temp, R', 
                                    'Vapor Molecular Weight lb/lb-mole', 
                                    'RVP, psia', 
                                    'Grant',
                                    'Constant A, unitless', 
                                    'Constant B, R'], 
              'parameter': ['liq_category', 'liq_name', 'tb', 'tla', 'tln', 'tlx', 'pva', 'pvn', 'pvx', 'mv', 'rvp', 'const_a', 'const_b'], 
              'value': [liq_category, liq_name, tb, tla, tln, tlx, pva, pvn, pvx, mv, rvp, const_a, const_b]})


### Calculation Details / Standing Losses

In [ ]:
ls = data[7]['data']['value']
vv = data[7]['data']['elements'][0]['vv']
wv= data[7]['data']['elements'][0]['wv']
ke= data[7]['data']['elements'][0]['ke']
ks= data[7]['data']['elements'][0]['ks']
print('-----------------------')
print('Calculation Details / Standing Losses')
print('ls: {} // vv: {} // wv: {} // ke: {} // ks: {}'.format(ls, vv, wv, ke, ks))

df_standing_losses = pd.DataFrame({'quantity': ['Standing Losses' for each in range(5)], 
                                  'parameter_name': ['Standing Losses, lbs', 
                                                     'Vapor Space Volume, ft^3', 
                                                     'Vapor Density, lb/ft^3', 
                                                     'Vapor Space Expansion Factor, 1/day', 
                                                     'Vented Vapor Saturation Factor, unitless'], 
                                     'parameter': ['ls', 'vv', 'wv', 'ke', 'ks'], 
                                      'value': [ls, vv, wv, ke, ks]})

### Working Losses

In [ ]:
lw = data[8]['data']['value']
vq = data[8]['data']['elements'][0]['vq']   
kn = data[8]['data']['elements'][0]['kn']   
kp = data[8]['data']['elements'][0]['kp']
wv = data[8]['data']['elements'][0]['wv']   
kb = data[8]['data']['elements'][0]['kb'] 
print('-----------------------')
print('Working Losses')
print('lw: {} // vq: {} // kn: {} // kp: {} // wv: {} // kb: {}'.format(lw, vq, kn, kp, wv, kb))

df_working_losses = pd.DataFrame({'quantity': ['Working Losses', 'Working Losses', 'Working Losses', 'Working Losses', 'Working Losses', 'Working Losses'], 
              'parameter_name': ['Working Losses, lbs/year', 
                                 'Net Working Loss Throughput, ft^3/year', 
                                 'Turnover Factor, unitless', 
                                 'Working Loss Product Factor, unitless', 
                                 'Vapor Density, lb/ft^3', 
                                 'Vent Setting Correction Factor, unitless'], 
              'parameter': ['lw', 'vq', 'kn', 'kp', 'wv', 'kb'], 
              'value': [lw, vq, kn, kp, wv, kb]})


### Total Losses

In [ ]:
lw = data[6]['data']['elements'][0]['lw'] 
ls = data[6]['data']['elements'][0]['ls'] 
lt = data[6]['data']['value']
print('-----------------------')
print('Total Losses')
print('lw: {} // ls: {} // lt: {}'.format(lw, ls, lt))

df_total_losses = pd.DataFrame({'quantity': ['Total Losses', 'Total Losses', 'Total Losses'], 
                                'parameter_name': ['working_loss, lbs', 'standing_loss, lbs', 'total_loss, lbs'], 
                                'parameter': ['lw', 'ls', 'lt'], 'value': [lw, ls, lt]})

### Vapor Space Volume

In [ ]:
vv = data[7]['data']['elements'][0]['vv']
diameter = data[8]['data']['elements'][0]['diameter']  
hvo = data[1]['data']['elements'][0]['hvo']
print('-----------------------')
print('Vapor Space Volume')
print('vv: {} //, diameter: {} // hvo: {}'.format(vv, diameter, hvo))

df_vapor_volume_space = pd.DataFrame({'quantity': ['Vapor Space Volume', 'Vapor Space Volume', 'Vapor Space Volume'], 
                                      'parameter_name': ['Vapor Space Volume', 'Tank Diameter', 'Vapor Space Outage'], 
                                      'parameter': ['vv', 'diameter', 'hvo'], 'value': [vv, diameter, hvo]})


### Vapor Space Outage

In [ ]:
hvo = data[1]['data']['elements'][0]['hvo']
hs = data[3]['data']['elements'][0]['shell height']
hl = data[3]['data']['elements'][0]['hl']
hro = data[3]['data']['elements'][0]['hro']
print('-----------------------')
print('Vapor Space Outage')
print('hvo: {} // hs: {} // hl: {} // hro: {}'.format(hvo, hs, hl, hro))

df_vapor_space_outage = pd.DataFrame({'quantity': ['Vapor Space Outage', 'Vapor Space Outage', 'Vapor Space Outage', 'Vapor Space Outage'], 'parameter_name': ['Vapor Space Outage', 
                                                 'Tank Shell Height', 
                                                 'Liquid Height', 
                                                 'Roof Outage'], 
                                'parameter': ['hvo', 
                                                                  'hs', 
                                                                  'hl', 
                                                                  'hro'], 
                          'value': [hvo, hs, hl, hro]})


### Roof Outage

In [ ]:
hro = data[3]['data']['elements'][0]['hro']
hr = data[3]['data']['elements'][0]['hr']
rs = data[3]['data']['elements'][0]['shell radius']
sr = data[3]['data']['elements'][0]['roof slope']
print('-----------------------')
print('Roof Outage')
print('hro: {} // hr: {} // rs: {} // sr: {}'.format(hro, hr, rs, sr))

df_roof_outage = pd.DataFrame({'quantity': ['Roof Outage', 
                                              'Roof Outage', 
                                              'Roof Outage', 
                                              'Roof Outage'], 
                                 
                                 'parameter_name': ['Roof Outage', 
                                                    'Tank Roof Height', 
                                                    'Tank Shell Radius', 
                                                    'Tank Cone Roof Slope'], 
                                 
                                 'parameter': ['hro', 
                                               'hr', 
                                               'rs', 
                                               'sr'], 
                                 
                                 'value': [hro, 
                                           hr, 
                                           rs, 
                                           sr]})

### Vapor Density

In [ ]:
wv = data[8]['data']['elements'][0]['wv']
mv = data[4]['data']['elements'][0]['mv']
pva = data[5]['data']['elements'][0]['pva']
r = data[4]['data']['elements'][0]['constant']
tv = data[4]['data']['elements'][0]['tv']
alpha_s = data[0]['data']['alphaS_shell']
alpha_r = data[0]['data']['alphaR_roof']
i_value = data[0]['data']['i_solarInsulation']
print('-----------------------')
print('Vapor Density')
print('wv: {} // mv: {} // pva: {} // r: {} // tv: {} // alpha_s: {} // alpha_r: {} // i_value: {} //'.format(wv, mv, pva, r, tv, alpha_s, alpha_r, i_value))

In [ ]:
df_vapor_density = pd.DataFrame({'quantity': ['Vapor Density', 
                                              'Vapor Density', 
                                              'Vapor Density', 
                                              'Vapor Density', 
                                              'Vapor Density', 
                                              'Vapor Density', 
                                              'Vapor Density', 
                                              'Vapor Density'], 
                                 
                                 'parameter_name': ['Vapor Density, lb/ft^3', 
                                                    'Vapor Molecular Weight, lb/lb-mole', 
                                                    'Vapor Pressure at Average Daily Liquid Surface Temp, psia',
                                                    'Ideal Gas Constant', 
                                                    'Average Vapor Temperature', 
                                                    'Tank Roof Surface Solar Absorptance', 
                                                    'Tank Shell Surface Solar Absorptance', 
                                                    'Average Daily Total Insolation Factor'], 
                                 
                                 'parameter': ['wv', 
                                               'mv', 
                                               'pva', 
                                               'r', 
                                               'tv', 
                                               'alpha_s', 
                                               'alpha_r', 
                                               'i_value'], 
                                 
                                 'value': [wv, 
                                           mv, 
                                           pva, 
                                           r, 
                                           tv, 
                                           alpha_s, 
                                           alpha_r, 
                                           i_value]})

### Vapor Space Expansion Factor

In [ ]:
ke= data[7]['data']['elements'][0]['ke']
del_tv = data[5]['data']['elements'][0]['delta_tv']
delta_pv = data[5]['data']['elements'][0]['delta_pv']
delta_pb = data[5]['data']['elements'][0]['delta_pb']
pa = data[5]['data']['elements'][0]['atm_pressure']
pva = data[5]['data']['elements'][0]['pva']
tla = data[5]['data']['elements'][0]['tla']
print('-----------------------')
print('Vapor Space Expansion Factor')
print('ke: {} // del_tv: {} // delta_pv: {} // delta_pb: {} //, pa: {} //, pva: {} //, tla: {} //'.format(ke, del_tv, delta_pv, delta_pb, pa, pva, tla))

### Vented Vapor Space Saturation Factor

In [ ]:
ks = data[2]['data']['value']
pva = data[2]['data']['elements'][0]['pva']
hvo = data[2]['data']['elements'][0]['hvo']
print('-----------------------')
print('Vented Vapor Space Saturation Factor')
print('ks: {} // pva: {} //  hvo: {}'.format(ks, pva, hvo))

In [ ]:
df_vented_vapor_space = pd.DataFrame({'quantity': ['Vented Vapor Space Saturation Factor', 
                                                     'Vented Vapor Space Saturation Factor', 
                                                     'Vented Vapor Space Saturation Factor', 
                                                     'Vapor Space Expansion Factor', 
                                                     'Vapor Space Expansion Factor', 
                                                     'Vapor Space Expansion Factor', 
                                                     'Vapor Space Expansion Factor', 
                                                     'Vapor Space Expansion Factor', 
                                                     'Vapor Space Expansion Factor', 
                                                     'Vapor Space Expansion Factor'], 
                    
                    'parameter_name': ['Vented Vapor Saturation Factor, unitless', 
                                       'Vapor Pressure at Average Daily Liquid Surface Temp, psia', 
                                       'Vapor Space Outage, ft', 
                                       'Vapor Space Expansion Factor, 1/day', 
                                       'Average Daily Vapor Presure Temp. Range, R', 
                                       'Average Daily Vapor Pressure Range, psi', 
                                       'Breather Vent Pressure Setting Range, psig', 
                                       'Atmospheric Pressure, psi', 
                                       'Vapor Pressure at Average Daily Liquid Surface Temp, psia', 
                                       'Average Daily Liquid Surface Temp, R'],
                    
                    'parameter': ['ks', 
                                  'pva', 
                                  'hvo', 
                                  'ke', 
                                  'del_tv', 
                                  'delta_pv', 
                                  'delta_pb', 
                                  'pa', 
                                  'pva', 
                                  'tla'], 
                    
                    'value': [ks, 
                              pva, 
                              hvo, 
                              ke, 
                              del_tv, 
                              delta_pv, 
                              delta_pb, 
                              pa, 
                              pva, 
                              tla]})

In [ ]:
list_df = [df_tank_dimensions,
           df_paint_characteristics,
           df_roof_characteristics,
           df_meterological_data,
           df_insulation_characteristics,
           df_liquid_data,
           df_standing_losses,
           df_vapor_volume_space,
           df_roof_outage,
           df_vapor_space_outage,
           df_breather_vent_settings,
           df_vapor_density, 
           df_vented_vapor_space,
           df_working_losses,
           df_total_losses]

df = pd.concat(list_df).reset_index(drop=True)

In [ ]:
from datetime import datetime

file_string = 'vfrt-output-' + datetime.today().strftime('%Y-%m-%d') + '.csv'
pd.concat(list_df).to_csv(file_string, index=False)

In [ ]:
df

In [ ]:
import requests as r
import json

POST_JSON = {
    'shellHeight':20, 
    'tankDiameter':15, 
    'tankDomeRoofRadius':9, 
    'breatherPressureSetting':0.03,
    'breatherVaccumSetting':-0.03,
    'tankLength':15,
    'throughput':10000, 
    'roofSlope':0.0625, 
    'averageLiquidHeight':10/2, 
    'rvp_crudeOils':5,
    'productMW':50,
    'isTankDomeRoofRadiusKnown':False, 
    'isTankOnVaporBalance':False, 
    'isRoofInsulated':False,
    'isShellInsulated':False,   
    'isAverageLiquidHeightKnown':True,
    'isCustomProduct':False,
    'roofType':'Cone',
    'tankOrientation':'Vertical', 
    'productType':'Crude Oils', 
    'breatherVentSetting':'Default',
    'location':'Long Beach, CA',
    'shellShade':'White', 
    'shellCondition':'Average', 
    'roofShade':'White',
    'roofCondition':'Average', 
    'productClassString':'Crude Oils',
    'productNameString':'Crude oil (RVP 5)', 
    'tankName':'Grant', 
    'assetNumber': '4209'}

url = 'https://tanks-409d-api.herokuapp.com/apiv001/vfrt'
response = r.post(url, json=POST_JSON)
data = response.text
data_json = json.loads(data)

In [ ]:
import pandas as pd

list(data_json.keys())

In [ ]:
data_json['standing_losses']